In [ ]:
from diffusers import DiffusionPipeline, DPMSolverMultistepScheduler
import torch
import random
from sdxl_service import SdxlService

: 

In [ ]:
sdxl_base_1_0_model_path = "./models//stable-diffusion-xl-base-1.0"
sdxl_refiner_1_0_model_path = "./models/stable-diffusion-xl-refiner-1.0"

In [ ]:
class NoWatermark:
    def apply_watermark(self, img):
        return img

In [ ]:
pipe = DiffusionPipeline.from_pretrained(
    sdxl_base_1_0_model_path,
    torch_dtype=torch.float16,
    variant="fp16",
    use_safetensors=True,
    local_files_only=True,
)
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
pipe.unet = torch.compile(pipe.unet, mode="reduce-overhead", fullgraph=True)
pipe.watermark = NoWatermark()
pipe.to("cuda")

In [ ]:
refiner = DiffusionPipeline.from_pretrained(
    sdxl_refiner_1_0_model_path,
    text_encoder_2=pipe.text_encoder_2,
    vae=pipe.vae,
    torch_dtype=torch.float16,
    variant="fp16",
    use_safetensors=True,
    local_files_only=True,
)
refiner.unet = torch.compile(refiner.unet, mode="reduce-overhead", fullgraph=True)
refiner.watermark = NoWatermark()
refiner.to("cuda")

In [ ]:
prompt = "An astronaut riding a green horse"
prompt = """
The word WAR in the background.
A crow sitting on sword stuck in a skull.
Blood moon in the background.

Zelda style.
Miyazaki style. 

Insanely detailed, 4k, hdr, anime. 
Intricate background, 
Old castle in the sky.
"""
prompt = """
Princess Peach riding a unicorn in the mushroom kingdom of Super Mario.
The Unicorn is laughing and happy.

Photorealistic, hyperrealistic, insanely detailed, 4k, hdr. 
Intricate background, national geographic award photography.
Elegance and serenity. Princess Peach has a symmetrical face.
"""
prompt = """
A giant billboard that says 'Peach you so cool' in large letter. 

Princess Peach in the mushroom kindgom. Bowser plays the grand piano. 

Night scene.

Zelda style.
Miyazaki style. 

Insanely detailed, 4k, hdr, anime. 
Intricate background, national geographic award photography.
"""
prompt = """
Crisp night in the Mushroom Kingdom. Huge billboard: 'Peach, you're so cool!'. Princess Peach is central, Bowser plays piano. Zelda and Miyazaki-style blend, like a high-definition, intricate 4K HDR anime. Background detail akin to a National Geographic winner."
"""
prompt = """
Satan playing guitar on a roof of a church.
contact lenses. blood moon in the background

inspired by Cyril Rolando, shutterstock, highly detailed illustration, full color illustration, very detailed illustration, dan mumford and alex grey style
"""
prompt1 = """
Satan playing Fender Stratocaster guitar on a roof of a church.
contact lenses, iris. blood moon in the background

HDR, award winning photography, 4k, hyperfocus, extremely detailed, hyperrealistic
"""
prompt2 = """
Jesus Christ playing Fender Stratocaster guitar on a roof of a church at sunrise. 
contact lenses, iris. rainbow in the background

HDR, award winning photography, 4k, hyperfocus, extremely detailed, hyperrealistic
"""

negative_prompt="closed eyes, epiphora, strabismus, Amblyopia, lazy eye, Polydactyly, Craniosynostosis, Plagiocephaly"


In [ ]:
prompt=prompt2

total = 4
height=640
# width=
width=1536
num_inference_steps=40
seed = random.randint(8, 10)
generator = torch.Generator(device="cuda").manual_seed(seed)

def generate_image(prompt):
    latent_images = pipe(
        prompt=prompt, 
        generator=generator, 
        output_type="latent", 
        height=height, 
        width=width, 
        num_inference_steps=num_inference_steps,
        negative_prompt=negative_prompt
    ).images
    image = refiner(prompt=prompt, generator=generator, image=latent_images).images[0]
    return image

images = []
for _ in range(total):
    image = generate_image(prompt)
    images.append(image)

for image in images:
    display(image)